In [7]:
import json
import pickle
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

In [32]:
data_dir = "./severity_dumps/"
filenames = os.listdir(data_dir)


['ta_joe_severity_r4',
 'ta_negin_severity_r5',
 'ta_sanjna_severity_r4_review',
 'ta_joe_severity_r1',
 'ta_xiaoyan_severity_r6',
 'ta_jillian_severity_r2',
 'ta_negin_severity_r4_review',
 'ta_xiaoyan_severity_r6_review',
 'ta_joe_severity_r2',
 'ta_xiaoyan_severity_r3',
 'ta_xiaoyan_severity_r5',
 'ta_sanjna_severity_r1',
 'ta_nour_severity_r1',
 'ta_joe_target_severity_r3',
 'ta_sanjna_severity_r3',
 'ta_sanjna_severity_r6_review',
 'ta_sanjna_severity_r2',
 'ta_mcgill_severity_r4_review',
 'ta_sanjna_severity_r4',
 'ta_nour_severity_r2',
 'ta_negin_severity_r1',
 'ta_sanjna_severity_r5',
 'ta_jillian_severity_r1',
 'ta_sanjna_severity_r6',
 'ta_negin_severity_r3',
 'ta_xiaoyan_severity_r1',
 'ta_joe_severity_training_r1',
 'ta_xiaoyan_severity_r4_review',
 'ta_xiaoyan_severity_r4',
 'ta_xiaoyan_severity_r2',
 'ta_joe_severity_r3',
 'ta_negin_severity_r4',
 'ta_negin_severity_r2']

In [60]:
def load_ds_by_keyword(data_dir, keyword, cols=['text', 'meta', 'label', 'accept', 'answer']):
    filenames = os.listdir(data_dir)
    filtered_fns = [name for name in filenames if keyword in name]
    datasets = {}
    for filename in filtered_fns:
        ds = pd.DataFrame(pickle.load(open(os.path.join(data_dir, filename), "rb")))
        datasets[filename] = ds
        print(ds.columns)
        print("{} has {} annotations".format(filename, len(datasets[filename])))
    return datasets

In [61]:
load_ds_by_keyword(data_dir, "r1")

Index(['text', 'meta', 'spans', 'label', 'accept', '_input_hash', '_task_hash',
       'options', '_session_id', '_view_id', 'answer'],
      dtype='object')
ta_joe_severity_r1 has 200 annotations
Index(['text', 'meta', 'spans', 'label', 'accept', '_input_hash', '_task_hash',
       'options', '_session_id', '_view_id', 'answer'],
      dtype='object')
ta_sanjna_severity_r1 has 172 annotations
Index(['text', 'meta', 'spans', 'label', 'accept', '_input_hash', '_task_hash',
       'options', '_session_id', '_view_id', 'answer'],
      dtype='object')
ta_nour_severity_r1 has 194 annotations
Index(['text', 'meta', 'spans', 'label', 'accept', '_input_hash', '_task_hash',
       'options', '_session_id', '_view_id', 'answer'],
      dtype='object')
ta_negin_severity_r1 has 200 annotations
Index(['text', 'meta', 'spans', 'label', 'accept', '_input_hash', '_task_hash',
       'options', '_session_id', '_view_id', 'answer'],
      dtype='object')
ta_jillian_severity_r1 has 200 annotations
Index

{'ta_joe_severity_r1':                                                   text  \
 0    Here we go again. Yet another LIBERAL accused ...   
 1    I remember when everyone was dickriding -JOHN ...   
 2    I'm a Matt Damon fan, can't even believe he wa...   
 3    Darn it. I was hoping a generation of tech emp...   
 4    Moore, <OTHER TARGET 2>, -JOHN DOE-, Mueller, ...   
 ..                                                 ...   
 195  -JOHN DOE- of Brand New and <OTHER TARGET 1> b...   
 196  As someone who watched an awful of Cartoon Net...   
 197        -JOHN DOE- looks like a nasty fuck and all.   
 198  Being the EIC of @Artforum is prestigious. Did...   
 199  Fact is examples like -JOHN DOE- are as common...   
 
                                                   meta  \
 0    {'tweet_id': '925856978475171841', 'mask_map':...   
 1    {'tweet_id': '930292240416731137', 'mask_map':...   
 2    {'tweet_id': '918439998898294784', 'mask_map':...   
 3    {'tweet_id': '921475052125

In [9]:
r1_filenames = [name for name in filenames if "r1" in name]
r1_filenames

['ta_joe_severity_r1',
 'ta_sanjna_severity_r1',
 'ta_nour_severity_r1',
 'ta_negin_severity_r1',
 'ta_jillian_severity_r1',
 'ta_xiaoyan_severity_r1',
 'ta_joe_severity_training_r1']

In [24]:
r1_datasets = {}
for filename in r1_filenames:
    
    r1_datasets[filename] = pickle.load(open(os.path.join(data_dir, filename), "rb"))
    print(filename, len(r1_datasets[filename]))

ta_joe_severity_r1 200
ta_sanjna_severity_r1 172
ta_nour_severity_r1 194
ta_negin_severity_r1 200
ta_jillian_severity_r1 200
ta_xiaoyan_severity_r1 145
ta_joe_severity_training_r1 0


In [18]:
r1_datasets.keys()

dict_keys(['ta_joe_severity_r1', 'ta_sanjna_severity_r1', 'ta_nour_severity_r1', 'ta_negin_severity_r1', 'ta_jillian_severity_r1', 'ta_xiaoyan_severity_r1', 'ta_joe_severity_training_r1'])

In [21]:
r1_datasets["ta_joe_severity_r1"][0].keys()


dict_keys(['text', 'meta', 'spans', 'label', 'accept', '_input_hash', '_task_hash', 'options', '_session_id', '_view_id', 'answer'])

In [23]:
r1_datasets["ta_joe_severity_r1"][0]["meta"]


{'tweet_id': '925856978475171841',
 'mask_map': {'<TARGET 1>': 'Michael Oreskes'},
 'pattern_match': True,
 'target_label': '<TARGET 1>',
 'target': 'Michael Oreskes'}

In [31]:
r1_datasets["ta_joe_severity_r1"][2]["accept"]


[1, '2']

# plan to clean
- [ ] go over rounds, on each round collect all items
- [ ] take majority vote (look into the paper and see what the did)
- [ ] based on tweet id, extract text, maj_vote, tweet_id